<a href="https://colab.research.google.com/github/ChaejinE/MyPytorch/blob/main/g_Model_save_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.onnx as onnx
import torchvision.models as models

# 모델 가중치 저장 및 불러오기
- PyTorch 모델은 학습한 매개변수를 state_dict라고 불리는 internal state dictionary에 저장한다.
- torch.save 메소드를 사용해 저장할 수 있다.

In [4]:
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), "model_weights.pth")

- 모델 가중치를 불러오려면 동일한 model instance를 생성한 다음 load_state_dict() 메소드를 사용하여 매개변수들을 불러온다
- model.eva() 메소드를 호출해 dropout과 batch normalization을 evaluation mode로 설정해야한다.
그렇지 않으면 추론 결과가 일관성이없어진다.

In [5]:
model = models.vgg16()
model.load_state_dict(torch.load("model_weights.pth"))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# 모델 형태를 포함하여 저장하고 불러오기
- 모델 가중치 불러올 때, NN의 구조를 정의하기 위해 모델 클래스를 먼저 생성해야 했다.
- 클래스의 구조를 모델과 함께 저장하고 싶으면 model.state_dict()가 아닌 model을 저장 함수에 전달한다.
- Python pickle 모듈을 사용해 모델을 직렬화하므로 모델을 불러올 때 실제 클래스 정의를 적용한다.

In [6]:
torch.save(model, "model.pth")

In [7]:
model = torch.load("model.pth")

# 모델을 ONNX로 내보내기
- PyTorch 실행 그래프의 동적 특성 때문에 내보내는 과정에 ONNX 모델을 생성하기 위해 실행 그래프를 탐색해야한다.
- 내보내기 단계에서는 적절한 크기의 테스트 변수를 전달해야한다.
  - 예시에서는 올바른 크기의 dummy 0 텐서를 생성
- [ONNX Tutorial](https://github.com/onnx/tutorials)

In [8]:
input_image = torch.zeros((1,3,244,244,))
onnx.export(model, input_image, "model.onnx")

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
